# beomi/KcELECTRA-base

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1586595974030012525
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2254123828
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15688572885698152456
physical_device_desc: "device: 0, name: GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [12]:
!pip install nbformat
!pip install nbconvert

In [11]:
import nbformat
from nbconvert import PythonExporter
from IPython.display import display, HTML

# 노트북 파일 경로
notebook_path = 'KcELECTRA_version.ipynb'

# 노트북 파일을 파이썬 코드로 변환
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

# 노트북 파일에서 파이썬 코드 추출
exporter = PythonExporter()
python_code, _ = exporter.from_notebook_node(nb)

# 추출한 파이썬 코드 실행
exec(python_code)

# 노트북 내용 출력
display(HTML("<h3>노트북 파일 내용:</h3>"))
display(nb)


라이브러리 설치가 완료되었습니다.


{'cells': [{'cell_type': 'code',
   'execution_count': 1,
   'metadata': {},
   'outputs': [],
   'source': 'import os\n\n# 파이썬 환경 구축 후 라이브러리 버전 맞춰 설치\npackages_to_install = [\n    "transformers==4.34.1",\n    "torch==2.1.0",\n    "soynlp==0.0.493",\n    "requests==2.31.0",\n    "tensorflow==2.10.0",\n    "accelerate==0.23.0",\n    "PyKoSpacing==0.5",\n    "kss==4.5.4",\n    "matplotlib==3.7.3",\n    "wordcloud==1.9.2"\n]\n\nfor package in packages_to_install:\n    os.system(f"pip install {package}")\n\n# pip 업그레이드\nos.system("python -m pip install --upgrade pip")\n\n# 파이썬 라이브러리 버전 전체 확인\nos.system("pip freeze")\n\n# 파이썬 라이브러리 버전 하나씩 확인\npackages_to_show = [\n    "transformers",\n    "torch",\n    "soynlp",\n    "requests",\n    "tensorflow",\n    "accelerate",\n    "PyKoSpacing",\n    "kss",\n    "matplotlib",\n    "wordcloud"\n]\n\nfor package in packages_to_show:\n    os.system(f"pip show {package}")\n\n# git에서 라이브러리 복제\nos.system("git clone https://github.com/ZIZUN/korean-malicious

In [3]:
import re
import torch
import pandas as pd

# deivce 선택
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("devcie:", device)

devcie: cpu


In [4]:
# 엑셀 파일에서 데이터프레임 읽기
df = pd.read_excel(r"C:\Users\GJAISCHOOL\Desktop\X_filter\알고리즘\욕 데이터\abc.xlsx")

# 'Sentence' 칼럼의 값을 문자열로 변환하여 리스트로 저장
str_data = df['Sentence'].astype(str).tolist()

# 'Sentence' 칼럼의 값을 하나의 문자열로 결합
all_sentences = ' '.join(str_data)

print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\GJAISCHOOL\\Desktop\\X_filter\\알고리즘\\욕 데이터\\abc.xlsx'

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud 

# 워드 클라우드 생성
wordcloud = WordCloud(width=1200, height=600, background_color='white').generate(all_sentences)

# 워드 클라우드 시각화
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
null_idx = df[df.label.isnull()].index                             # 해당 index에 null 값 확인
df.loc[null_idx, "Sentence"]                                       # null 값이 존재한 인덱스의 content 값 불러오기

# lable은 content의 가장 끝 문장열로 설정
df.loc[null_idx, "label"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열로 설정
df.loc[null_idx, "Sentence"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-2])

In [ ]:
train_data = df.sample(frac=0.8, random_state=42)                 # train(80%), test(20%) 셋 구분 
test_data = df.drop(train_data.index)                             # 랜덤으로 샘플링(랜덤으로 숫자 배치)

# 데이터셋 갯수 확인
print('중복 제거 전 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋 : {}'.format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=["Sentence"], inplace=True)
test_data.drop_duplicates(subset=["Sentence"], inplace=True)

# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))

In [ ]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, TFElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# KcELECTRA 모델 및 토크나이저 로드
model_name = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenized_train_sentences = tokenizer(
    list(train_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [ ]:
tokenized_test_sentences = tokenizer(
    list(test_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [ ]:
print(tokenized_train_sentences[0])        # 0번째 문장에 해당하는 객체 
print(tokenized_train_sentences[0].tokens) # 0번째 문장에 토큰의 목록
print(tokenized_train_sentences[0].ids)    # 0번째 문장에 대한 고유 ID

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encoding, labels):
        self.encodings = encoding
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여
        item["labels"] = torch.tensor(self.labels[idx])                             # key: toch.tensor(val[ix])라는 item 딕셔너리 형성
        return item                                                                 # 새로운 labels 키 값에 value torch.tensor(self.labels[idx]) 쌍을 생성
    
    def __len__(self):
        return len(self.labels)                                                     # self.labels 길이 반환

In [ ]:
# train_set, test_set에 대한 데이터셋을 각각 생성

train_label = train_data["label"].values
test_label = test_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)   # 사전 학습된 모델 찾아오기
model.to(device)

In [ ]:
# 3 - 2 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir = './results',           # 학습결과 저장경로
    num_train_epochs=10,                # 학습 epoch 설정
    per_device_train_batch_size = 8,    # train batch_size 설정
    per_device_eval_batch_size = 64,    # test batch_size 설정
    logging_dir = './logs',             # 학습 log 저장경로
    logging_steps=100,                  # 학습 log 기록 단위
    save_total_limit = 2,               # 학습결과 저장 최대갯수
)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 학습과정에서 사용할 평가지표를 위한 함수 설정
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # 정밀도, 재현율, f1 구하기 
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  # 정확도 구하기
  acc = accuracy_score(labels, preds)
  return{
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

In [ ]:
# Trainer 모듈을 사용해 모델의 학습을 컨트롤하은 trainer를 생성
trainer = Trainer(
    model = model,                       # 학습하고자하는 Transformers model
    args=training_args,                  # 위에서 정의한 Trainging Arguments
    train_dataset=train_dataset,         # 학습 데이터셋
    eval_dataset=test_dataset,           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 평가지표
)

In [ ]:
trainer.train()

In [ ]:
# 평가를 수행하고 custom_ 접두사가 붙은 평가 지표를 출력
results = trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix='custom_')

print(results)

In [ ]:
# 0: curse, 1: non_curse
def sentence_predict(sent):
    # 평가모드로 변경
    model.eval()

    # 입력된 문장 토크나이징
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    # 모델이 위치한 GPU로 이동
    # tokenized_sent.to(deivce)

    # 예측
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
        )

    # 결과 return
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    sentence = True
    input_sentence = tokenized_sent["input_ids"]
    if result == 0:
        sentence = True

    elif result == 1:
        sentence = False
    return sentence, input_sentence


sentence_predict("씨발이게 맞아?")

In [ ]:
def found_word(input_sentence, found_bad_word):
    result = input_sentence
    if found_bad_word:
        result = badword_find(result)
    return result

def badword_find(input_sentence):
    result = input_sentence
    badword_df = pd.read_excel(r'C:\Users\GJAISCHOOL\Desktop\X_filter\알고리즘\욕 데이터\word_list.xlsx')
    
    found_bad_word = False  # 입력 문장에 단어가 발견되었는지를 나타내는 플래그
    for idx, row in badword_df.iterrows():
        if row["WORD"] in input_sentence:
            # 'WORD'가 입력 문장에 포함된 경우
            new_word = row["대체어"]
            if not pd.isnull(new_word):
                result = result.replace(row["WORD"], new_word)
                found_bad_word = True
                break  # 대체어를 찾았으므로 반복문 종료
            else:
                # result = result.replace(row["WORD"], "*" * len(row["WORD"]))
                result = "욕설을 사용했습니다."
                found_bad_word = True
    
    if not found_bad_word:
        # result = "@" * len(input_sentence)
        result = "혐오 표현입니다."
    return result

# 테스트
input_sentence = "아가리"
speak_result = badword_find(input_sentence)
print(speak_result)

In [ ]:
badword_find(input_sentence)

In [ ]:
def speak(sent):
    sentence = sentence_predict(sent)
    if sentence[0] == False:
        return sent
    elif sentence[0] == True:
        return badword_find(sent)

In [ ]:
import kss
from pykospacing import Spacing
from soynlp.normalizer import repeat_normalize

# 띄어쓰기 설정
spacing = Spacing()

In [ ]:
# 특수문자 제거
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ|a-zA-Z0-9]','', text)
    return text

In [ ]:
def clean_and_repeat_normalize(text):
    cleansed_text = cleanse(text)                                             # 특수문자 제거
    normalized_text = repeat_normalize(cleansed_text, num_repeats=2)          # 중복문자 제거
    input_data = re.sub(r'\d', '', normalized_text)                           # 숫자 제거
    normalized_text = spacing(input_data)                                     # 띄어쓰기 보정 

    return normalized_text

clean_and_repeat_normalize("어바지가방에들어가신다")

In [ ]:
def final_output():
    input_text = clean_and_repeat_normalize(input())
    sentences = kss.split_sentences(input_text)

    sentences_list = []
    for sentence in sentences:
        text_output = speak(sentence)
        sentences_list.append(text_output)

    long_test = ' '.join(sentences_list)
    print(long_test)
    return long_test

In [ ]:
final_output()